In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import math
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [17]:
housing_data = pd.read_csv('housing.csv')
housing_data.head()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
0,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
1,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
2,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
3,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
4,0.02985,0.0,2.18,0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7


#EDA

In [18]:
housing_data.describe()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
count,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000,505.000000
mean,3.620663,11.350495,11.154257,0.069307,0.554728,6.284059,68.581584,3.794460,9.566337,408.459406,18.461782,356.594376,12.668257,22.529901
std,8.608569,23.343704,6.855868,0.254227,0.115990,0.703195,28.176371,2.107761,8.707553,168.629992,2.162520,91.367787,7.139950,9.205991
min,0.009060,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082210,0.000000,5.190000,0.000000,0.449000,5.885000,45.000000,2.100000,4.000000,279.000000,17.400000,375.330000,7.010000,17.000000
50%,0.259150,0.000000,9.690000,0.000000,0.538000,6.208000,77.700000,3.199200,5.000000,330.000000,19.100000,391.430000,11.380000,21.200000
75%,3.678200,12.500000,18.100000,0.000000,0.624000,6.625000,94.100000,5.211900,24.000000,666.000000,20.200000,396.210000,16.960000,25.000000
max,88.976000,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.127000,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [19]:
housing_data.corr()

,0.00632,18,2.31,0,0.538,6.575,65.2,4.09,1,296,15.3,396.9,4.98,24
0.00632,1.000000,-0.200283,0.406251,-0.056132,0.420934,-0.218978,0.352701,-0.379626,0.625395,0.582568,0.289393,-0.384838,0.455328,-0.388249
18,-0.200283,1.000000,-0.534022,-0.042550,-0.516574,0.311835,-0.569524,0.664396,-0.311717,-0.314351,-0.391713,0.175319,-0.412894,0.360393
2.31,0.406251,-0.534022,1.000000,0.062350,0.764556,-0.391330,0.645543,-0.708848,0.594167,0.720561,0.380955,-0.356506,0.602737,-0.484126
0,-0.056132,-0.042550,0.062350,1.000000,0.091134,0.091497,0.086461,-0.099109,-0.007907,-0.035965,-0.122570,0.049040,-0.054576,0.175364
0.538,0.420934,-0.516574,0.764556,0.091134,1.000000,-0.302127,0.731461,-0.769220,0.611758,0.668141,0.188918,-0.380006,0.591262,-0.427295
6.575,-0.218978,0.311835,-0.391330,0.091497,-0.302127,1.000000,-0.240211,0.205170,-0.209277,-0.291680,-0.355116,0.127754,-0.613734,0.695365
65.2,0.352701,-0.569524,0.645543,0.086461,0.731461,-0.240211,1.000000,-0.747872,0.456232,0.506527,0.261724,-0.273486,0.602782,-0.376932
4.09,-0.379626,0.664396,-0.708848,-0.099109,-0.769220,0.205170,-0.747872,1.000000,-0.494797,-0.534492,-0.232560,0.291451,-0.497276,0.249895
1,0.625395,-0.311717,0.594167,-0.007907,0.611758,-0.209277,0.456232,-0.494797,1.000000,0.910202,0.463322,-0.444065,0.487608,-0.381690
296,0.582568,-0.314351,0.720561,-0.035965,0.668141,-0.291680,0.506527,-0.534492,0.910202,1.000000,0.460100,-0.441505,0.543435,-0.468543


In [79]:
class LinearRegression:

  def __init__(self, X, y, learningrate, tolerance, maxIteration = 50000, error = 'rmse', gradient_descent = False):
    self.X = X
    self.y = y
    self.learningrate = learningrate
    self.maxIteration = maxIteration
    self.error = error
    self.gradient_descent = gradient_descent
    self.tolerance = tolerance

  def split_data(self, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)
    return X_train, X_test, y_train, y_test
  
  def addX0(self, X):
    """
    @X: numpy matrix, dataset
    @does: add a bias term to the data
    @return: numpy matrix 
    """
    return np.column_stack([np.ones([X.shape[0], 1]), X])

  def normalize(self, X):
    """
    @X: numpy matrix, dataset
    @does: normalize the X and not the target value (y)
    @return: numpy matrix, normalized dataset
    """
    mean = np.mean(X, 0) # zero because we want mean of each column
    standard_deviation = np.std(X, 0)
    norm_X = (X - mean)/standard_deviation
    norm_X = self.addX0(norm_X)
    return norm_X, mean, standard_deviation

  def normalize_test(self, X, trainMean, trainStd):
    """
    @X: numpy matrix, dataset
    @trainMean: numpy matrix, mean of training set
    @trainStd: numpy matrix, standard deviation of training set
    @does: normalize the X with respect to training data (because we don't touch test set)
    @return: numpy matrix, normalized dataset
    """
    norm_X = (X - trainMean)/trainStd
    norm_X = self.addX0(norm_X)
    return norm_X
    
  def getRank(self, X, epsilon = 1e-12):
    """
    @X: numpy matrix, Dataset
    @epsilon: float, threshold for SVD
    @does: checks the rank of matrix and performs SVD
    @return: length of values of S which are greater than epsilon
    """
    u, S, vh = np.linalg.svd(X)
    return len([x for x in S if x > epsilon])

  def checkMatrix(self, X):
    """
    @X: numpy matrix, Dataset
    @does: if x_rank = min(rows, columns) then matrix is full rank
    """
    x_rank = self.getRank(X)
    if (x_rank == min(X.shape[0], X.shape[1])):
      self.full_rank = True
      print('Matrix is a full rank matrix.')
    else:
      self.full_rank = False
      print('Matrix is not a full rank matrix.')

  def checkInvertible(self, X):
    if X.shape[0] < X.shape[1]:
      self.low_rank = True
      print('Matrix is a low rank matrix.')
    else:
      self.low_rank = False
      print('Matrix is not a low rank matrix.')

  def closedFormSolution(self, X, y):
    """
    @X: numpy matrix, Dataset
    @y: numpy vector, target value
    @does: solve regression using closed form solution -> (X^T.X)^-1.X.y
    @return: numpy array, parameters
    """
    return np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

  def predict(self, X):
    return X.dot(self.w)

  def standard_square_error(self, X, y):
    """
    @return: cost function without the regularization term
    """
    y_hat = self.predict(X) #Q is y_hat (note that Q is theta)
    return (((y_hat - y)**2).sum())

  def rmse(self, X, y):
    """
    @does: we divide by the total records we have and then square root it.
    """
    return math.sqrt(self.standard_square_error(X, y) / y.size)

  def cost_function(self, X, y):
    return self.standard_square_error(X, y)/2

  def cost_derivative(self, X, y):
    y_hat = self.predict(X)
    return (y_hat - y).dot(X) # 2^XTQX - 2X^Ty (note that here Q is theta and QX is y_hat)

  def gradientDescent(self, X, y):
    error_sequences = [] #an empty error which has errors for every iteration
    previous_error = float('inf') #initialization with infinity
    
    for i in tqdm(range(self.maxIteration)):
      self.w  = self.w - self.learningrate * self.cost_derivative(X, y)
      if self.error == 'rmse':
        current_error = self.rmse(X, y)
      else:
        current_error = self.standard_square_error(X, y)

      difference_error = previous_error - current_error
      previous_error = current_error

      if difference_error < self.tolerance:
        print("The model has stopped. No further improvement.")
        break
      
    return

  def trainModel(self):
    self.X_train, self.X_test, self.y_train, self.y_test = self.split_data(self.X, self.y)

    #Normalize the data
    self.X_train, self.mean, self.standard_deviation = self.normalize(self.X_train)
    #Normalize test data
    self.X_test = self.normalize_test(self.X_test, self.mean, self.standard_deviation)

    self.checkMatrix(self.X_train)
    self.checkInvertible(self.X_train)

    # closed form approach
    if self.full_rank and not self.low_rank and self.X_train.shape[0] < 10000 and not self.gradient_descent:
      self.w = self.closedFormSolution(self.X_train, self.y_train)

    # else gradient descent
    else:
      self.w = np.ones(self.X_train.shape[1], dtype = np.float64) * 0
      self.gradientDescent(self.X_train, self.y_train)

    print(self.w)
    if self.error == 'rmse':
      print(self.rmse(self.X_test, self.y_test))
    else:
      print(self.standard_square_error(self.X_test, self.y_test))

In [80]:
regression = LinearRegression(housing_data.values[:, 0:-1], housing_data.values[:, -1],
                              learningrate = 0.000001, tolerance = 0.000001,
                              gradient_descent = True, error = 'rmse')

In [81]:
regression.trainModel()

  5%|▍         | 2350/50000 [00:00<00:02, 23492.07it/s]

Matrix is a full rank matrix.
Matrix is not a low rank matrix.


 62%|██████▏   | 31243/50000 [00:00<00:00, 31958.39it/s]

The model has stopped. No further improvement.
[22.93384277 -0.81929069  0.87191772  0.05130278  0.70684921 -1.78427617
  2.75467927  0.2942418  -2.45287459  1.38580546 -0.78592131 -2.24195901
  0.65609747 -3.87998616]
4.650938822823211
